# Loss Visualization

This notebook visualizes loss curves obtained from the training of latent models or finetuning of LoRAdapters.

If the csv files are not available, they can be extracted from Tensorboard using the `tensorboard_export.ipynb` notebook.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

## LatentVAE

In [ ]:
# Load losses for the other model versions
df_v5 = pd.read_csv('losses/latent_vae_v5_val_losses.csv')
df_v8 = pd.read_csv('losses/latent_vae_v8_val_losses.csv')
df_v16 = pd.read_csv('losses/latent_vae_v16_val_losses.csv')
df_v17 = pd.read_csv('losses/latent_vae_v17_val_losses.csv')
df_v18 = pd.read_csv('losses/latent_vae_v18_val_losses.csv')
df_v19 = pd.read_csv('losses/latent_vae_v19_val_losses.csv')
df_v20 = pd.read_csv('losses/latent_vae_v20_val_losses.csv')

# Preprocess disc loss (multiply with disc_active)
df_v16['val/disc_loss'] = df_v16['val/disc_loss'] * df_v16['val/disc_active']
df_v17['val/disc_loss'] = df_v17['val/disc_loss'] * df_v17['val/disc_active']
df_v18['val/disc_loss'] = df_v18['val/disc_loss'] * df_v18['val/disc_active']
df_v19['val/disc_loss'] = df_v19['val/disc_loss'] * df_v19['val/disc_active']
df_v20['val/disc_loss'] = df_v20['val/disc_loss'] * df_v20['val/disc_active']

# List of losses to plot
loss_dict = {
	'val/rec_lat_loss': 'Latent Reconstruction Loss',
	'val/rec_img_loss': 'Image Reconstruction Loss',
	'val/perc_img_loss': 'Perceptual Image Loss',
	'val/kl_loss': 'KL Divergence Loss',
	'val/disc_loss': 'Discriminator Loss',
	'val/gen_loss': 'Generator Loss',
	'val/total_loss': 'Total Loss'
}

# Create one subplot per loss
n_cols = 2
n_rows = math.ceil(len(loss_dict) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 4 * n_rows), sharex=True)
axes = axes.flatten()

# Collect handles and labels for the shared legend
handles_dict = {}

for ax, (loss, loss_name) in zip(axes[:-1], loss_dict.items()):  # Exclude the last axis
    if loss in df_v5.columns:
        line, = ax.plot(df_v5['step'], df_v5[loss], color='C0', label='16k → 128 (VAE Loss)')
        handles_dict['16k → 128 (VAE Loss)'] = line
    if loss in df_v8.columns:
        line, = ax.plot(df_v8['step'], df_v8[loss], color='C1', label='16k → 128 (VAE+Disc. Loss)')
        handles_dict['16k → 128 (VAE+Disc. Loss)'] = line
    line, = ax.plot(df_v16['step'], df_v16[loss], color='C2', label='16k → 512')
    handles_dict['16k → 512'] = line
    line, = ax.plot(df_v17['step'], df_v17[loss], color='C3', label='16k → 512 (Alt. Design)')
    handles_dict['16k → 512 (Alt. Design)'] = line
    line, = ax.plot(df_v19['step'], df_v19[loss], color='C4', label='16k → 4k')
    handles_dict['16k → 4k'] = line
    line, = ax.plot(df_v18['step'], df_v18[loss], color='C5', label='16k → 8k')
    handles_dict['16k → 8k'] = line
    line, = ax.plot(df_v20['step'], df_v20[loss], color='C6', label='16k → 8k (L1 Loss)')
    handles_dict['16k → 8k (L1 Loss)'] = line
    ax.grid(True)
    ax.set_title(loss_name, fontsize=16)

# Add shared legend to the last axes
handles = list(handles_dict.values())
labels = list(handles_dict.keys())
axes[-1].legend(handles, labels, loc='center', fontsize=16)
axes[-1].axis('off')  # Hide the plot area for the last axes

# shared x-labels for all axes
axes[-2].set_xlabel('Steps')
axes[-3].set_xlabel('Steps')
axes[-3].tick_params(axis='x', labelbottom=True)


plt.suptitle('Comparison of Losses Across VAE Versions', fontsize=20, y=1.02)
plt.tight_layout()
# plt.savefig('vis/latent_vae_losses_comparison.pdf', bbox_inches='tight')
plt.show()

## LatentVAE, LatentAE, LatentLinearAE

In [ ]:
# Load losses
df_vae = pd.read_csv('losses/latent_vae_v16_val_losses.csv')
df_ae = pd.read_csv('losses/latent_ae_v1_val_losses.csv')
df_linear_ae = pd.read_csv('losses/latent_linear_ae_v4_val_losses.csv')

# Preprocess disc loss (multiply with disc_active)
df_vae['val/disc_loss'] = df_vae['val/disc_loss'] * df_vae['val/disc_active']
df_ae['val/disc_loss'] = df_ae['val/disc_loss'] * df_ae['val/disc_active']
df_linear_ae['val/disc_loss'] = df_linear_ae['val/disc_loss'] * df_linear_ae['val/disc_active']

# List of losses to plot
loss_dict = {
	'val/rec_lat_loss': 'Latent Reconstruction Loss',
	'val/rec_img_loss': 'Image Reconstruction Loss',
	'val/perc_img_loss': 'Perceptual Image Loss',
	'val/fid_score': 'Fréchet Inception Distance',
}

# Create one subplot per loss
n_cols = 2
n_rows = math.ceil(len(loss_dict) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 4 * n_rows), sharex=True)
axes = axes.flatten()

# Add losses
for ax, (loss, loss_name) in zip(axes, loss_dict.items()):  # Exclude the last axis
    line, = ax.plot(df_vae['step'], df_vae[loss], color='C0', label='LatentVAE')
    line, = ax.plot(df_ae['step'], df_ae[loss], color='C1', label='LatentAE')
    line, = ax.plot(df_linear_ae['step'], df_linear_ae[loss], color='C2', label='LatentLinearAE')
    ax.grid(True)
    ax.legend(loc='upper right')
    ax.set_title(loss_name, fontsize=16)

plt.suptitle('Validation Loss Comparison of LatentVAE, LatentAE, and LatentLinearAE', fontsize=20, y=1.02)
plt.tight_layout()
# plt.savefig('vis/latent_vae_ae_linearae_losses_comparison.pdf', bbox_inches='tight')
plt.show()

## LatentVQVAE, LatentVQVAE2, VQVAE2

In [ ]:
# Load losses
df_lvqvae_v7 = pd.read_csv('losses/latent_vqvae_v7_val_losses.csv')
df_lvqvae_v8 = pd.read_csv('losses/latent_vqvae_v8_val_losses.csv')
df_lvqvae2_v0 = pd.read_csv('losses/latent_vqvae2_v0_val_losses.csv')
df_lvqvae2_v1 = pd.read_csv('losses/latent_vqvae2_v1_val_losses.csv')
df_vqvae2_v0 = pd.read_csv('losses/vqvae2_v0_val_losses.csv')

# Preprocess disc loss (multiply with disc_active)
df_lvqvae_v7['val/disc_loss'] = df_lvqvae_v7['val/disc_loss'] * df_lvqvae_v7['val/disc_active']
df_lvqvae_v8['val/disc_loss'] = df_lvqvae_v8['val/disc_loss'] * df_lvqvae_v8['val/disc_active']
df_lvqvae2_v0['val/disc_loss'] = df_lvqvae2_v0['val/disc_loss'] * df_lvqvae2_v0['val/disc_active']
df_lvqvae2_v1['val/disc_loss'] = df_lvqvae2_v1['val/disc_loss'] * df_lvqvae2_v1['val/disc_active']
df_vqvae2_v0['val/disc_loss'] = df_vqvae2_v0['val/disc_loss'] * df_vqvae2_v0['val/disc_active']

# List of losses to plot
loss_dict = {
	'val/rec_lat_loss': 'Latent Reconstruction Loss',
	'val/rec_img_loss': 'Image Reconstruction Loss',
	'val/perc_img_loss': 'Perceptual Image Loss',
    'val/codebook_loss': 'Codebook Loss',
    'val/perplexity': 'Perplexity',
	'val/fid_score': 'Fréchet Inception Distance',
}

# Create one subplot per loss
n_cols = 2
n_rows = math.ceil(len(loss_dict) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 4 * n_rows), sharex=True)
axes = axes.flatten()

# Add losses
for ax, (loss, loss_name) in zip(axes, loss_dict.items()):  # Exclude the last axis
    line, = ax.plot(df_lvqvae_v7['step'], df_lvqvae_v7[loss], color='C0', label='LatentVQVAE')
    line, = ax.plot(df_lvqvae_v8['step'], df_lvqvae_v8[loss], color='C1', label='LatentVQVAE + larger codebook')
    line, = ax.plot(df_lvqvae2_v0['step'], df_lvqvae2_v0[loss], color='C2', label='LatentVQVAE2')
    line, = ax.plot(df_lvqvae2_v1['step'], df_lvqvae2_v1[loss], color='C3', label='LatentVQVAE2 + larger latent space')
    line, = ax.plot(df_vqvae2_v0['step'], df_vqvae2_v0[loss], color='C4', label='VQVAE2')
    ax.grid(True)
    ax.legend(loc='upper right')
    ax.set_title(loss_name, fontsize=16)

plt.suptitle('Validation Loss Comparison of LatentVQVAE, LatentVQVAE2, and VQVAE2', fontsize=20, y=1.02)
plt.tight_layout()
# plt.savefig('vis/latent_vqvae_vqvae2_losses_comparison.pdf', bbox_inches='tight')
plt.show()

## Latent Prior

In [ ]:
# Load losses
df_v11_val = pd.read_csv('losses/latent_prior_v11_val_losses.csv')
df_v11_train = pd.read_csv('losses/latent_prior_v11_train_losses.csv')
df_v16_val = pd.read_csv('losses/latent_prior_v16_val_losses.csv')
df_v16_train = pd.read_csv('losses/latent_prior_v16_train_losses.csv')

# List of losses to plot
loss_dict = {
	'loss_bottom': 'Bottom Loss',
	'loss_top': 'Top Loss',
	'total_loss': 'Total Loss',
}

# Create one subplot per loss
n_cols = 2
n_rows = math.ceil(len(loss_dict) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 4 * n_rows), sharex=True)
axes = axes.flatten()

# Add losses
for ax, (loss, loss_name) in zip(axes, loss_dict.items()):  # Exclude the last axis
    ax.plot(df_v11_val['step'], df_v11_val['val/'+loss], color='C0', label='VQVAE2 Prior (Val)')
    ax.plot(df_v11_train['step'], df_v11_train['train/'+loss], color='C0', linestyle='--', label='VQVAE2 Prior (Train)')
    ax.plot(df_v16_val['step'], df_v16_val['val/'+loss], color='C1', label='LatentVQVAE2 Prior (Val)')
    ax.plot(df_v16_train['step'], df_v16_train['train/'+loss], color='C1', linestyle='--', label='LatentVQVAE2 Prior (Train)')
    ax.grid(True)
    ax.legend(loc='upper right')
    ax.set_title(loss_name, fontsize=16)

# Remove last axis
axes[-1].axis('off')

plt.suptitle('Loss Comparison of Hierarchical Transformer Prior in VQVAE2 and LatentVQVAE2 Latent Space', fontsize=20, y=1.02)
plt.tight_layout()
# plt.savefig('vis/latent_prior_losses_comparison.pdf', bbox_inches='tight')
plt.show()

## LoRA Fine-Tuning

In [ ]:
# Load losses
df_lora_v7 = pd.read_csv('losses/sd_lora_v7_val_losses.csv')
df_lora_v8 = pd.read_csv('losses/sd_lora_v8_val_losses.csv')
df_lora_v9 = pd.read_csv('losses/sd_lora_v9_val_losses.csv')

# List of losses to plot
loss_dict = {
	'val/loss': 'Loss',
	'val/fid_score': 'Fréchet Inception Distance',
	'val/spectral_score': 'Spectral Difference',
}

# Create one subplot per loss
n_cols = 2
n_rows = math.ceil(len(loss_dict) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 4 * n_rows), sharex=True)
axes = axes.flatten()

# Add losses
handles_dict = {}
for ax, (loss, loss_name) in zip(axes, loss_dict.items()):  # Exclude the last axis
    line, = ax.plot(df_lora_v9['step'], df_lora_v9[loss], color='C0', label='Style Adapter')
    handles_dict['Style Adapter'] = line
    line, = ax.plot(df_lora_v7['step'], df_lora_v7[loss], color='C1', label='Style+Depth Adapter')
    handles_dict['Style+Depth Adapter'] = line
    line, = ax.plot(df_lora_v8['step'], df_lora_v8[loss], color='C2', label='Style+HED Adapter')
    handles_dict['Style+HED Adapter'] = line
    ax.grid(True)
    ax.set_title(loss_name, fontsize=16)

# Add shared legend to the last axes
handles = list(handles_dict.values())
labels = list(handles_dict.keys())
axes[-1].legend(handles, labels, loc='center', fontsize=16)
axes[-1].axis('off')  # Hide the plot area for the last axes

# shared x-labels for all axes
axes[-2].set_xlabel('Steps')
axes[-3].set_xlabel('Steps')
axes[-3].tick_params(axis='x', labelbottom=True)

plt.suptitle('Validation Losses of LoRAdapter Finetuning', fontsize=20, y=1.02)
plt.tight_layout()
# plt.savefig('vis/ex4_adapter_finetuning_losses.pdf', bbox_inches='tight')
plt.show()